<a href="https://colab.research.google.com/github/mangeshb1/nlp_experiments/blob/main/Llama_2_GGML_CTransformers/Llama_2_GGML_CTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Nov 28 00:21:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4399.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd i

In [3]:
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ctransformers: filename=ctransformers-0.2.27-cp310-cp310-linux_x86_64.whl size=2338830 sha256=eaddc6b20810139b8f1b328dc2c158f68607ca3e49c48e292306160302ead21e
  Stored in directory: /root/.cache/pip/wheels/dd/54/e9/32364da8eee84a2b0b412394983c15add18816c507e90f02d8
Successfully built ctransformers


In [4]:
# choose your champion
#model_id = "TheBloke/Llama-2-7B-GGML"
#model_id = "TheBloke/Llama-2-7B-chat-GGML"
#model_id = "TheBloke/Llama-2-13B-GGML"
model_id = "TheBloke/Llama-2-13B-chat-GGML"

In [5]:
from ctransformers import AutoModelForCausalLM

config = {'max_new_tokens': 256, 'repetition_penalty': 1.1, 'temperature': 0.1, 'stream': True}

llm = AutoModelForCausalLM.from_pretrained(model_id,
                                           model_type="llama",
                                           #lib='avx2', for cpu use
                                           gpu_layers=130, #110 for 7b, 130 for 13b
                                           **config
                                           )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-13b-chat.ggmlv3.q2_K.bin:   0%|          | 0.00/5.51G [00:00<?, ?B/s]

## LLAMA-2-7B-Chat tests

### Tokenizer

In [6]:
prompt="""Write a poem to help me remember the first 10 elements on the periodic table, giving each
element its own line."""

In [7]:
tokens = llm.tokenize(prompt)

In [8]:
tokens

[1,
 6113,
 263,
 26576,
 304,
 1371,
 592,
 6456,
 278,
 937,
 29871,
 29896,
 29900,
 3161,
 373,
 278,
 29591,
 1591,
 29892,
 6820,
 1269,
 13,
 5029,
 967,
 1914,
 1196,
 29889]

### Pipeline

In [9]:
# 'pipeline' execution
llm(prompt, stream=False)

"\n\nI'm trying to learn the periodic table and I need some help!\n\nHere is what I have so far:\n\nHydrogen (H)\nHelium (He)\nLithium (Li)\nBeryllium (Be)\nBoron (B)\nCarbon (C)\nNitrogen (N)\nOxygen (O)\nFluorine (F)\nNeon (Ne)\n\nI can't seem to remember the rest of the elements!  Can you help me?\n\nHere is what I am trying to memorize:\n\n1. Hydrogen (H)\n2. Helium (He)\n3. Lithium (Li)\n4. Beryllium (Be)\n5. Boron (B)\n6. Carbon (C)\n7. Nitrogen (N)\n8. Oxygen (O)\n9. Fluorine (F)\n10. Neon (Ne)\n\nI hope this helps! Let me know if you have any other questions.\n\nBest,\n[Your Name]"

In [10]:
prompt2 = """Quando e por quem o Brasil foi descoberto?"""
llm(prompt2, stream=False)

'\n\nA descoberta do Brasil é um evento histórico que ocorreu em 1500, quando a expedição de Pedro Álvares Cabral alcançou as costas do atual Brasil. A expedição partiu da Portugal em 2 de abril de 1500 e chegou ao Brasil em 22 de maio do mesmo ano.\n\nA descoberta do Brasil é creditada a Pedro Álvares Cabral, que foi o líder da expedição. Ele era um navegador português que havia participado de outras viagens para o leste da África e para a Índia. A expedição de Cabral foi a primeira a chegar ao Novo Mundo desde a viagem de Cristóvão Colombo em 1492.\n\nA descoberta do Brasil teve um impacto significativo na história do mundo, pois marcou o início da colonização europeia das Américas e ajudou a estabelecer o domínio português sobre a região.'

### Generate with stream execution

In [11]:
# LlAMA-2-7b-chat execution
import time
start = time.time()
NUM_TOKENS=0
print('-'*4+'Start Generation'+'-'*4)
for token in llm.generate(tokens):
    print(llm.detokenize(token), end='', flush=True)
    NUM_TOKENS+=1
time_generate = time.time() - start
print('\n')
print('-'*4+'End Generation'+'-'*4)
print(f'Num of generated tokens: {NUM_TOKENS}')
print(f'Time for complete generation: {time_generate}s')
print(f'Tokens per secound: {NUM_TOKENS/time_generate}')
print(f'Time per token: {(time_generate/NUM_TOKENS)*1000}ms')

----Start Generation----


I'm trying to learn the periodic table and I need some help!

Here is what I have so far:

Hydrogen (H)
Helium (He)
Lithium (Li)
Beryllium (Be)
Boron (B)
Carbon (C)
Nitrogen (N)
Oxygen (O)
Fluorine (F)
Neon (Ne)

I can't seem to remember the rest of the elements!  Can you help me?

Here is what I am trying to memorize:

1. Hydrogen (H)
2. Helium (He)
3. Lithium (Li)
4. Beryllium (Be)
5. Boron (B)
6. Carbon (C)
7. Nitrogen (N)
8. Oxygen (O)
9. Fluorine (F)
10. Neon (Ne)

I hope this helps! Let me know if you have any other questions.

Best,
[Your Name]

----End Generation----
Num of generated tokens: 247
Time for complete generation: 20.9249050617218s
Tokens per secound: 11.804115682791808
Time per token: 84.71621482478463ms


## LLAMA-2-13B-Chat tests

### Tokenizer

In [12]:
prompt="""Write a poem to help me remember the first 10 elements on the periodic table, giving each
element its own line."""

In [13]:
tokens = llm.tokenize(prompt)

In [14]:
tokens

[1,
 6113,
 263,
 26576,
 304,
 1371,
 592,
 6456,
 278,
 937,
 29871,
 29896,
 29900,
 3161,
 373,
 278,
 29591,
 1591,
 29892,
 6820,
 1269,
 13,
 5029,
 967,
 1914,
 1196,
 29889]

### Pipeline

In [15]:
# 'pipeline' execution
llm(prompt, stream=False)

'\n\nI\'m trying to learn these elements for my chemistry class and I need some help\nmemorizing them!\n\nHere is what I have so far:\n\nHydrogen (H)\nHelium (He)\nLithium (Li)\nBeryllium (Be)\nBoron (B)\nCarbon (C)\nNitrogen (N)\nOxygen (O)\nFluorine (F)\nNeon (Ne)\n\nI would appreciate any help you can give me! Thank you!"\n\nOf course, I\'d be happy to help! Here is a poem to help you remember the first 10 elements on the periodic table:\n\nHydrogen\'s light and helium\'s flight\nLithium\'s bright, beryllium\'s might\nBoron\'s strong, carbon\'s right\nOxygen\'s air, fluorine\'s fight\nNeon\'s glow, a brilliant sight\n\nI hope this poem helps you remember the first 10 elements on the periodic table! Just think of the words and the associations they evoke, and you should be'

In [16]:
prompt2 = """Quando e por quem o Brasil foi descoberto?"""
llm(prompt2, stream=False)

'\n\nA descoberta do Brasil é um evento histórico que ocorreu em 1500, quando a expedição de Pedro Álvares Cabral alcançou as costas do atual Brasil. A expedição partiu da Portugal em 2 de abril de 1500 e chegou ao Brasil em 22 de fevereiro de 1500.\n\nA descoberta do Brasil é creditada a Pedro Álvares Cabral, que foi o líder da expedição. Ele era um navegador português que havia participado de outras viagens para o leste da África e para a Índia. A expedição de Cabral foi a primeira a chegar ao Novo Mundo desde a viagem de Cristóvão Colombo em 1492.\n\nA descoberta do Brasil teve um impacto significativo na história do mundo, pois marcou o início da colonização europeia das Américas e ajudou a estabelecer o domínio português sobre a região. A descoberta também levou ao estabelecimento de relações comerciais entre a Portugal'

### Stream generation

In [17]:
# LlAMA-2-13b-chat execution
import time
start = time.time()
NUM_TOKENS=0
print('-'*4+'Start Generation'+'-'*4)
for token in llm.generate(tokens):
    print(llm.detokenize(token), end='', flush=True)
    NUM_TOKENS+=1
time_generate = time.time() - start
print('\n')
print('-'*4+'End Generation'+'-'*4)
print(f'Num of generated tokens: {NUM_TOKENS}')
print(f'Time for complete generation: {time_generate}s')
print(f'Tokens per secound: {NUM_TOKENS/time_generate}')
print(f'Time per token: {(time_generate/NUM_TOKENS)*1000}ms')

----Start Generation----


I'm trying to learn these in order to pass my chemistry class.

Please make it rhyme and be easy to remember."

Here is a poem to help you remember the first 10 elements on the periodic table:

Hydrogen is number one, a gas that's light as air
Helium comes next, and it's rare
Lithium follows, with a charge so bright
Beryllium's next, and it's quite tight

Boron's the next one, a mineral to find
Carbon's the one that makes all life twind
Nitrogen's essential, for growth so fine
Oxygen's the one that keeps us in line

I hope this poem helps you remember the first 10 elements on the periodic table! Each element is given its own line, and the rhyme scheme is used to make it easy to remember. Good luck with your chemistry class!

----End Generation----
Num of generated tokens: 211
Time for complete generation: 16.92486262321472s
Tokens per secound: 12.466866331345294
Time per token: 80.21261906736835ms


In [21]:
len(tokens)

27

In [32]:
prompt="""Itinerary for 3 day trip in Mumbai ?"""

tokens = llm.tokenize(prompt)

In [ ]:
# LlAMA-2-13b-chat execution
import time
start = time.time()
NUM_TOKENS=0
print('-'*4+'Start Generation'+'-'*4)
for token in llm.generate(tokens):
    print(llm.detokenize(token), end='', flush=True)
    NUM_TOKENS+=1
time_generate = time.time() - start
print('\n')
print('-'*4+'End Generation'+'-'*4)
print(f'Num of generated tokens: {NUM_TOKENS}')
print(f'Time for complete generation: {time_generate}s')
print(f'Tokens per secound: {NUM_TOKENS/time_generate}')
print(f'Time per token: {(time_generate/NUM_TOKENS)*1000}ms')

----Start Generation----

Mumbai, formerly known as Bombay, is the financial and entertainment capital of India. It is a city that never sleeps and has something to offer for everyone. Here's an itinerary for a 3-day trip to Mumbai:

Day 1:

* Morning: Start your day by visiting the Gateway of India, one of the most iconic landmarks of Mumbai. Take a ferry ride from here to Elephanta Island and explore the ancient Hindu and Buddhist caves.
* Afternoon: Head to the Marine Drive, also known as the Queen's Necklace, which is a beautiful promenade along the sea. You can take a walk or enjoy the sunset views.
* Evening: Visit the Colaba Causeway, a popular shopping destination that offers everything from street food to high-end boutiques. Enjoy dinner at one of the many restaurants in this area.

Day 2:

* Morning: Start your day by visiting the iconic Taj Mahal Hotel, which is located in the heart of Mumbai. Take a tour of the hotel and enjoy the stunning views of the city from its rooftop

 in

 the

la

ke

...

le

is

le

is

le

## LangChain Integration

https://python.langchain.com/docs/ecosystem/integrations/ctransformers